# BERT

At todays meeting, we're going to focus on BERT -- a model, which transforms our tokens into contextual word embeddings, that are of great quality!

We will use `transformers` library for that purpose, so let's install it first:

---
**Done by:** Sofya Aksenyuk, 150284.

---

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.0 MB/s eta 0:00:00


#BERT

## Task 1: Tokenization using WordPiece

Below you can find a simple fragment of code, which downloads a `bert-base` model (an `uncased` version -- this model introduced a preprocessing step on the data that transformed each text into lowercased text) and instantiates appropriate tokenizer for that model. You can learn about this particular model here: https://huggingface.co/bert-base-uncased (I encourage you to read the description! Many models hosted on the huggingface website have great documentations and it's always worth checking them).

Then, in line 4, we define a text to be tokenized, run the tokenizer in line 5, and use the tokenized input as the input to BERT model, which is called in line 6. 

Please, run the code below. We generated BERT embeddings using 6 lines of code! ;)

In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Runnning tokenizer by simply calling the tokenizer object (`tokenizer(text, return_tensors='pt')`) returns a dictionary containing 3 keys: `input_ids`, `token_type_ids`, and `attention_mask`. As for now, let's focus on `input_ids` only. 

You may visit the website: https://huggingface.co/docs/transformers/glossary to learn more about the role of `token_type_ids` and `attention_mask`.


The `input_ids` is a list of lists (represented as a tensor). The outer list collects documents, while the inner lists collect tokens in that document. Here, we processed only one document (sentence) so that there is only one "outer" list. 

Each of the inner lists contains a seqeunce of identifiers. These are the positions of tokens in the Vocabulary. They can be used to generate one-hot encoding representations for our words, since if we know the length of a vocabulary and know the position of a given token in the vocabulary, we can generate a vector of the vocabulary length that is filled with zeros, then we set the value assigned to a token position to 1 to generate one-hot encoding.

To produce the ids, we have to tokenize our text first. 

Moreover, those identifiers require less memory that storing tokens as strings!

Run the code below, to see the tokenizer's output. Please note that the number of token identifiers generated by the tokenizer is not equal to the number of tokens in the sequence. We'll see why is that in a minute.


In [3]:
encoded_input

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

The ids generated by the tokenizer are not human readable. However, the tokenizer contains a mapping which relates known tokens to their positions, so that we can use it to map ids to tokens back!

The first line of code obtains the input ids defined for the first sequence provided. 
Then, we call `convert_ids_to_tokens` to transform those ids to tokens. Please run the code and analyze the output.

In [4]:
first_sentence_ids = encoded_input['input_ids'][0]
tokenizer.convert_ids_to_tokens(first_sentence_ids)

['[CLS]',
 'replace',
 'me',
 'by',
 'any',
 'text',
 'you',
 "'",
 'd',
 'like',
 '.',
 '[SEP]']

Whoa, we see that the tokenizer not only tokenizes our text (splits into tokens), but it also generated special tokens required by our model! Nice. Please note that after tokenization we don't have capital letters in our tokens. This is caused by the use of the `bert-base-uncased` model. Since the model was trained on lowercased data, the tokenizer also ensures that the tokens are lowercased.


## Subword units

However, the length of the generated `input_ids` may be even bigger as related to the length of our text. Sometimes, the vocabulary doesn't contain a given word as a whole. Since BERT used WordPiece tokenization, to handle such cases, the tokenizer tries to split those words into smaller fragments, that are stored in our vocabulary. Let's tokenize some document containing rare words and see the result.

In [5]:
tokenizer_output = tokenizer(['NVIDIA DGX A100'])

input_ids = tokenizer_output['input_ids'][0]
input_ids

[101, 1050, 17258, 2401, 1040, 2290, 2595, 17350, 8889, 102]

Whoa, we see that as a result of tokenization, we obtained many tokens. Let's see what they represent: 

In [6]:
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', 'n', '##vid', '##ia', 'd', '##g', '##x', 'a1', '##00', '[SEP]']

Ah, as we can see, the vocabulary assigned to `bert-base` doesn't contain tokens such as "nvidia", "dgx" and "a100", that's why they are split into subwrod units. 

Each time a given subword unit starts with a double hash (##), we know that this subword unit is a continuation of the previous token. 

We can use this information to reconstruct the original text, by joining those subword units (sometimes called subtokens) into full tokens. We can achieve this goal using the following line of code:

In [7]:
tokenizer.decode(input_ids)

'[CLS] nvidia dgx a100 [SEP]'

If we know how to map tokens to their positions in the vocabulary, the only missing part is to determine how long our one-hot-encoding vector should be (or how big is our vocabulary). 

This transformation of ids into one-hot-encodings is done automatically by the `transformer` library. However, you can check the size of the vocabulary easily using the following line of code:

In [8]:
tokenizer.vocab_size

30522

## Task 2: Using a pre-trained BERT to generate features that may be used to solve a classification task.

As we discussed during the lecture, we can generate a fixed-length vector represening any input by taking the embedding produced for the `[CLS]` token (which is a representation of the whole seqeunce).

In this task, your goal is to use a pre-trained BERT model to obtain representations for a given dataset. Then, these representations will be used to train a logistic regression model. 

We will try to genearate a solution that can detect whether a given review is positive or not!

For that purpose we're going to use a BERT variant called `distilBERT`. We didn't cover it during the lecture, because it is related to the concept of distillation that will be very important to us and I'll make a separate lecture on it. 

For now, we can treat `distilBERT` the same way as BERT. In fact, this is a compressed BERT model. It behaves the same way but it is distilled so that we aimed to achieve similar quality using less parameters.

Please follow the tutorial you can find here: https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb (there is even a blogpost related to this tutorial: http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)


Just copy and paste fragments of code here and observe the results:

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [10]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
batch_1 = df[:2000]

batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

In [11]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [13]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

(2000, 59)

In [14]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 59)

In [15]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [16]:
features = last_hidden_states[0][:,0,:].numpy()
labels = batch_1[1]

In [17]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [18]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.8266666666666668


In [19]:
lr_clf = LogisticRegression(C=5.2)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=5.2)

In [20]:
lr_clf.score(test_features, test_labels)

0.792

In [21]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.527 (+/- 0.00)


## Task 3(Optional - not required): fine-tuning BERT

In the example above, a BERT model (or more precisely it's smaller family member: distilBERT) was used only to deliver vectors representing whole documents. 

Now we would like to fine-tune an existing model. As we discussed during the lecture, we can achieve it by simply switching the top-layer of the network. Instead of solving the Masked Language Model and Next Sentence Prediction tasks, we may add our own classification layer (also referred to as a classification head) and train our whole network to solve a given task.

There is a great and easy to follow tutorial on fine-tuning, available here: https://github.com/huggingface/notebooks/blob/main/transformers_doc/training.ipynb 

If you want, please follow that tutorial (you can copy-and-paste the code snippets from the notebook here).

In [22]:
#@title
from IPython.display import HTML

HTML('')

In [24]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 19.8 MB/s eta 0:00:00


In [25]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [26]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [27]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [28]:
#@title
from IPython.display import HTML

HTML('')

In [29]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [30]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [31]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [32]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [33]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Sorry, it requires 5h to train the model... But the tutorial is fine!